In [1]:
import azureml.core
azureml.data.__version__

'1.25.0'

# Import packages

In [2]:
# import packages
from datetime import datetime, timedelta

from azureml.core import Dataset, Workspace

In [4]:
ws = Workspace.from_config()
dstore = ws.get_default_datastore()

dset_name = 'weather-data-florida'

# Download data

In [9]:
import os

weather_data_folder = os.path.join(os.getcwd(), 'weather-data/2019')
os.makedirs(weather_data_folder, exist_ok=True)

In [19]:
import urllib3

http = urllib3.PoolManager()
for index in range (1, 9):
    mount_path = os.path.join(os.getcwd(), f'weather-data/2019/0{index}')
    os.makedirs(mount_path, exist_ok=True)

    file_url = f'https://github.com/Azure/MachineLearningNotebooks/raw/master/how-to-use-azureml/work-with-data/datasets-tutorial/timeseries-datasets/weather-data/2019/0{index}/data.parquet'
    response = http.request('GET', file_url)

    # Write data to file
    filename = f'{mount_path}/data.parquet'
    file_ = open(filename, 'wb')
    file_.write(response.data)
    file_.close()
    response.release_conn()

# Upload the data to storage

In [20]:
dstore.upload('weather-data', dset_name, overwrite=True, show_progress=True)

Uploading an estimated of 10 files
Uploading weather-data/.DS_Store
Uploaded weather-data/.DS_Store, 1 files out of an estimated total of 10
Uploading weather-data/2019/.DS_Store
Uploaded weather-data/2019/.DS_Store, 2 files out of an estimated total of 10
Uploading weather-data/2019/03/data.parquet
Uploaded weather-data/2019/03/data.parquet, 3 files out of an estimated total of 10
Uploading weather-data/2019/04/data.parquet
Uploaded weather-data/2019/04/data.parquet, 4 files out of an estimated total of 10
Uploading weather-data/2019/05/data.parquet
Uploaded weather-data/2019/05/data.parquet, 5 files out of an estimated total of 10
Uploading weather-data/2019/02/data.parquet
Uploaded weather-data/2019/02/data.parquet, 6 files out of an estimated total of 10
Uploading weather-data/2019/07/data.parquet
Uploaded weather-data/2019/07/data.parquet, 7 files out of an estimated total of 10
Uploading weather-data/2019/08/data.parquet
Uploaded weather-data/2019/08/data.parquet, 8 files out of 

$AZUREML_DATAREFERENCE_a9c95c81b19742fcb7b8dbc899ffb3c8

# Create and register tabular dataset

In [23]:
datastore_path = [(dstore, dset_name + '/*/*/data.parquet')]
dataset = Dataset.Tabular.from_parquet_files(path=datastore_path, partition_format = dset_name + '/{partition_time:yyyy/MM}/data.parquet')

tsd = dataset.with_timestamp_columns(timestamp='datetime', partition_timestamp='partition_time')

registered_ds = tsd.register(ws, dset_name, create_new_version=True, description='Data for Tabular Dataset with time-series trait demo.', tags={ 'type': 'TabularDataset' })

In [ ]:
# Continue here
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/work-with-data/datasets-tutorial/timeseries-datasets/tabular-timeseries-dataset-filtering.ipynb